# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

*course assignment instructions in italics*

## Part 1 - Scraping Data from Wikipedia Pages

for the scraping I used a tutorial from [simpleanalytical.com](https://simpleanalytical.com/how-to-web-scrape-wikipedia-python-urllib-beautiful-soup-pandas)

##### Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

*For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.*

*Start by creating a new Notebook for this assignment.
Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe:*

In [4]:
# import the library we use to open URLs
import urllib.request

In [5]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#open the url using urllib.request and put the HTML into the page variable
page = urllib.request.urlopen(url)

In [6]:
# import the BeautifulSoup library so we can parse HTML and XML documents
from bs4 import BeautifulSoup

In [7]:
# parse the HTML from our URL into the BeautifulSoup parse tree format
soup = BeautifulSoup(page, "lxml")

In [194]:
# let's look at the html code
#print(soup.prettify()) # <- uncomment to have a look at it

There we have it. Our table starts with **`<table class="wikitable">`**

let's fetch the table...

In [39]:
# fetch our table
wikiTable=soup.find('table', class_='wikitable')
wikiTable

<table class="wikitable">
<tbody><tr>
<th>Postal code
</th>
<th>Borough
</th>
<th>Neighborhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park / Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor / Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park / Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern / Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3B
</td>
<td>North York
</td>
<td>Don Mills
</td></tr>
<tr>
<td>M4B
</td>
<td>East York
<

In [31]:
# import pandas to be able to work with dataframes
import pandas as pd

- *The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood*

In [148]:
# define the dataframe columns
columnNames = ['PostalCode', 'Borough', 'Neighborhood'] 

# instantiate the dataframe
dfPC = pd.DataFrame(columns=columnNames)

- *Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.*
- *If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.*
- *More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will - notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.*

In [185]:
# fill wiki table in a dataframe
A=[]
B=[]
C=[]
    
for row in wikiTable.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        tmpPostalCode=cells[0].find(text=True).rstrip()
        tmpBorough=cells[1].find(text=True).rstrip()
        tmpNeighborhood=cells[2].find(text=True).rstrip()
        tmpNeighborhood=tmpNeighborhood.replace(' /',',')   # two rows will be combined into 
                                                            # one row with the neighborhoods separated with a comma
        if tmpBorough!='Not assigned': # Ignore cells with a borough that is Not assigned
            A.append(tmpPostalCode)
            B.append(tmpBorough)
            if tmpNeighborhood=='': # cell has a borough but a Not assigned neighborhood
                C.append(tmpBorough) # neighborhood will be the same as the borough
            else:
                C.append(tmpNeighborhood)

so now let's fill the contents of the list into a dataframe

In [191]:
df=pd.DataFrame({columnNames[0]: A, columnNames[1]: B, columnNames[2]: C})
df.head(15)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


- *In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.*

In [192]:
df.shape

(103, 3)

**So we have a dataframe with 103 unique rows and 3 columns.**

## Part 2 - Making Use of geospatial Data